In [35]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import MeanShift

In [36]:
import csv
# been found on the Internet
with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [37]:
data = pd.read_csv("checkins.csv")

In [38]:
data.shape

(396634, 6)

In [39]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [45]:
test_data = data.iloc[:100000, :].loc[:, ['latitude', 'longitude']]
test_data.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [46]:
#launch clustering
clustering = MeanShift(bandwidth=0.1).fit(test_data)
clustering

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [47]:
labels = clustering.labels_
cl_centres = clustering.cluster_centers_
print('N_clusters : %d' % len(cl_centres))

N_clusters : 3230


In [60]:
unique_labels, labels_count = np.unique(labels, return_counts = True)
set_of_labels = []
for index, cnt in enumerate(labels_count):
    if cnt > 15:
        set_of_labels.append(unique_labels[index])
print('Counts more than 15: %d ;' % len(set_of_labels))        

clusters_for_labels = np.empty((len(set_of_labels), 2))
for idx, lbl in enumerate(set_of_labels):
    clusters_for_labels[idx] = cl_centres[lbl]
print(clusters_for_labels)

Counts more than 15: 592 ;
[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  38.65877915  -76.8856871 ]
 [  39.2494686   -77.1821271 ]]


In [61]:
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

In [71]:
coord = pd.read_csv("coord.csv", skipinitialspace=True)
coord

,laptitude,longtitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [104]:
the_nearest = np.empty(len(set_of_labels))
for inx, item in enumerate(clusters_for_labels):
    min_dist = distance(item, coord.loc[0])
    for j in range(len(coord)):
        dist = distance(item, coord.loc[j])
        if dist < min_dist:
            min_dist = dist
    the_nearest[inx] = min_dist

In [107]:
print ("min dist coord =", clusters_for_labels[np.argsort(the_nearest)][0])
print ("min dist =", sorted(the_nearest)[0]) 


min dist coord = [-33.86063043 151.20477593]
min dist = 0.007834758163109155
